In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.2.6
setup Colab for PySpark 3.2.3 and Spark NLP 4.2.6


In [2]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

:: loading settings :: url = jar:file:/Users/maziyar/anaconda3/envs/sparknlp/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/maziyar/.ivy2/cache
The jars for the packages stored in: /Users/maziyar/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-074434f8-be75-400c-9b86-3fd89d7cbdf4;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.6 in central
	found com.typesafe#config;1.4.2 in spark-list
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in spark-list
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in spark-list
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in spark-list
	found com.google.code.gson#gson;2.3 in spark-list
	found it.unimi.dsi#fastutil;7.0.12 in spark-list
	found org.projectlombok#lombok;1.16.8 in spark-list
	found com.google.cloud#google-cloud-storage;2.15.0 in central
	found com.google.guava#guava;31.1-jre in central
	foun

In [3]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.train
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.testa

In [4]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng.train')

training_data.show(3)

22/12/29 13:48:05 WARN TaskSetManager: Stage 0 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[{document, 0, 47...|[{document, 0, 47...|[{token, 0, 1, EU...|[{pos, 0, 1, NNP,...|[{named_entity, 0...|
|     Peter Blackburn|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 4, Pe...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
| BRUSSELS 1996-08-22|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 7, BR...|[{pos, 0, 7, NNP,...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [5]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [6]:
word2Vec = Word2VecApproach()\
    .setInputCols("token")\
    .setOutputCol("embeddings")\
    .setMaxSentenceLength(1000)\
    .setStepSize(0.025)\
    .setMinCount(5)\
    .setVectorSize(100)\
    .setNumPartitions(1)\
    .setMaxIter(1)\
    .setSeed(42)\
    .setStorageRef("word2vec_conll03")\

nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(1)\
    .setLr(0.003)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setValidationSplit(0.2)\
    .setOutputLogsPath('ner_logs')  # if not set, logs will be written to ~/annotator_logs
#    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
#    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch

ner_pipeline = Pipeline(stages=[
    word2Vec,
    nerTagger
])


In [7]:
ner_model = ner_pipeline.fit(training_data)

22/12/29 13:48:07 WARN TaskSetManager: Stage 1 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.
22/12/29 13:48:09 WARN TaskSetManager: Stage 3 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.
22/12/29 13:48:10 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/29 13:48:10 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/29 13:48:14 WARN TaskSetManager: Stage 6 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.
22/12/29 13:48:18 WARN TaskSetManager: Stage 7 contains a task of very large size (9058 KiB). The maximum recommended task size is 1000 KiB.
2022-12-29 13:48:27.921143: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) 

Training started - total epochs: 1 - lr: 0.003 - batch size: 8 - labels: 9 - chars: 83 - training examples: 11265
Epoch 1/1 started, lr: 0.003, dataset size: 11265
Epoch 1/1 - 171.09s - loss: 3970.4763 - batches: 1411
Quality on validation dataset (20.0%), validation examples = 2253
time to finish evaluation: 11.93s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 1175	 263	 217	 0.8171071	 0.8441092	 0.8303887
I-ORG	 458	 141	 207	 0.76460767	 0.6887218	 0.7246835
I-MISC	 104	 45	 136	 0.6979866	 0.43333334	 0.5347043
I-LOC	 132	 29	 101	 0.8198758	 0.5665236	 0.6700508
I-PER	 867	 148	 32	 0.8541872	 0.9644049	 0.90595615
B-MISC	 480	 94	 205	 0.83623695	 0.7007299	 0.76250994
B-ORG	 868	 236	 339	 0.7862319	 0.7191384	 0.75119
B-PER	 1189	 321	 138	 0.78741723	 0.89600605	 0.8382094
tp: 5273 fp: 1277 fn: 1375 labels: 8
Macro-average	 prec: 0.7954563, rec: 0.72662085, f1: 0.7594821
Micro-average	 prec: 0.80503815, rec: 0.79317087, f1: 0.79906046


In [8]:
!cd ./ner_logs && ls -l

total 8
-rw-r--r--  1 maziyar  staff  1016 Dec 29 13:51 NerDLApproach_56719b1fca3b.log


In [9]:
!cat ./ner_logs/{nerTagger.uid}.log

Name of the selected graph: ner-dl/blstm_10_100_128_120.pb
Training started - total epochs: 1 - lr: 0.003 - batch size: 8 - labels: 9 - chars: 83 - training examples: 11265


Epoch 1/1 started, lr: 0.003, dataset size: 11265


Epoch 1/1 - 171.09s - loss: 3970.4763 - batches: 1411
Quality on validation dataset (20.0%), validation examples = 2253
time to finish evaluation: 11.93s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 1175	 263	 217	 0.8171071	 0.8441092	 0.8303887
I-ORG	 458	 141	 207	 0.76460767	 0.6887218	 0.7246835
I-MISC	 104	 45	 136	 0.6979866	 0.43333334	 0.5347043
I-LOC	 132	 29	 101	 0.8198758	 0.5665236	 0.6700508
I-PER	 867	 148	 32	 0.8541872	 0.9644049	 0.90595615
B-MISC	 480	 94	 205	 0.83623695	 0.7007299	 0.76250994
B-ORG	 868	 236	 339	 0.7862319	 0.7191384	 0.75119
B-PER	 1189	 321	 138	 0.78741723	 0.89600605	 0.8382094
tp: 5273 fp: 1277 fn: 1375 labels: 8
Macro-average	 prec: 0.7954563, rec: 0.72662085, f1: 0.7594821
Micro-average	 prec: 0.80503815, rec: 0.79317087,

In [10]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, './eng.testa')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|CRICKET - LEICEST...|[{document, 0, 64...|[{document, 0, 64...|[{token, 0, 6, CR...|[{pos, 0, 6, NNP,...|[{named_entity, 0...|
|   LONDON 1996-08-30|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 5, LO...|[{pos, 0, 5, NNP,...|[{named_entity, 0...|
|West Indian all-r...|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 3, We...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



22/12/29 13:51:33 WARN TaskSetManager: Stage 8 contains a task of very large size (1773 KiB). The maximum recommended task size is 1000 KiB.


In [11]:
predictions = ner_model.transform(test_data)

In [12]:
import pyspark.sql.functions as F

from sklearn.metrics import classification_report

preds_df = predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                                     predictions.label.result,
                                                     predictions.ner.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()

print (classification_report(preds_df['ground_truth'], preds_df['prediction']))

22/12/29 13:51:36 WARN TaskSetManager: Stage 9 contains a task of very large size (1773 KiB). The maximum recommended task size is 1000 KiB.
2022-12-29 13:51:36.821957: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AddV2: CPU 
AssignSub: CPU 
RealDiv: CPU 
Shape: CPU 
Unique: CPU 
Cast: CPU 
UnsortedSegmentSum: CPU 
Add: CPU 
GatherV2: CPU 
St

              precision    recall  f1-score   support

       B-LOC       0.82      0.85      0.83      1837
      B-MISC       0.87      0.69      0.77       922
       B-ORG       0.80      0.68      0.74      1341
       B-PER       0.78      0.90      0.84      1842
       I-LOC       0.76      0.56      0.65       257
      I-MISC       0.75      0.53      0.62       346
       I-ORG       0.79      0.58      0.67       751
       I-PER       0.86      0.96      0.91      1307
           O       0.99      0.99      0.99     42759

    accuracy                           0.96     51362
   macro avg       0.82      0.75      0.78     51362
weighted avg       0.96      0.96      0.96     51362



## Save and Restore


### Annotator Models
Let's say you would like to only save the trained annotators inside your pipeline so you can load them inside another custom Pipeline

In [13]:
# all we need is to access that stage and save it on disk
ner_model.stages

[Word2VecModel_039e5d98c036, NerDLModel_b873e071b194]

In [14]:
print(ner_model.stages[-1])
print(ner_model.stages[-2])

NerDLModel_b873e071b194
Word2VecModel_039e5d98c036


In [15]:
# let's save our NerDLModel - let's mention it was trained by word2vec_conll03 as well
ner_model.stages[-1].write().overwrite().save("./nerdl_conll03_word2vec_model")

In [16]:
# and here is our trained Word2VecModel
ner_model.stages[-2].write().overwrite().save("./word2vec_conll03_model")

This is how you use your saved model within your pipeline

In [17]:
document = DocumentAssembler().setInputCol("text").setOutputCol("document")

sentence = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

token = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

word2vecModel = (
    Word2VecModel.load("./word2vec_conll03_model")
    .setInputCols("token")
    .setOutputCol("embeddings")
)

nerdlModel = (
    NerDLModel.load("./nerdl_conll03_word2vec_model")
    .setInputCols(["sentence", "token", "embeddings"])
    .setOutputCol("ner")
)

ner_prediction_pipeline = Pipeline(
    stages=[document, sentence, token, word2vecModel, nerdlModel]
)


2022-12-29 13:51:50.782856: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AddV2: CPU 
AssignSub: CPU 
RealDiv: CPU 
Shape: CPU 
Unique: CPU 
Cast: CPU 
UnsortedSegmentSum: CPU 
Add: CPU 
GatherV2: CPU 
StridedSlice: CPU 
Identity: CPU 
Fill: CPU 
NoOp: CPU 
RandomUniform: CPU 
Mul: CPU 
Sub: CPU 
Sqrt: CPU 
Assign: CPU 
VariableV2: CPU 
Scatte

In [18]:
# or you can use it via DataFrame
from pyspark.sql.types import StringType

dfTest = spark.createDataFrame([
    "My name is John and I am a Doctor in London!",
    "Peter Parker is a nice persn and lives in New York. Bruce Wayne is also a nice guy and lives in Gotham city."
], StringType()).toDF("text")

ner_prediction_pipeline\
  .fit(dfTest)\
  .transform(dfTest)\
  .select("ner.result")\
  .show(2, False)

+-------------------------------------------------------------------------------------------------------+
|result                                                                                                 |
+-------------------------------------------------------------------------------------------------------+
|[O, O, O, B-PER, O, O, O, O, B-LOC, O, B-LOC, I-LOC]                                                   |
|[B-PER, I-PER, O, O, O, O, O, O, O, B-LOC, I-LOC, O, B-PER, I-PER, O, O, O, O, O, O, O, O, B-LOC, O, O]|
+-------------------------------------------------------------------------------------------------------+



### Save and restore the whole Pipeline

In [19]:

ner_prediction_pipeline.write().overwrite().save("./ner_conll03_word2vec_pipeline")
# let's load it back and try
loadedPipeline = Pipeline.load("./ner_conll03_word2vec_pipeline")
loadedPipeline.stages
# we have all of our stages inside the loaded pipeline!

2022-12-29 13:51:59.049574: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AddV2: CPU 
AssignSub: CPU 
RealDiv: CPU 
Shape: CPU 
Unique: CPU 
Cast: CPU 
UnsortedSegmentSum: CPU 
Add: CPU 
GatherV2: CPU 
StridedSlice: CPU 
Identity: CPU 
Fill: CPU 
NoOp: CPU 
RandomUniform: CPU 
Mul: CPU 
Sub: CPU 
Sqrt: CPU 
Assign: CPU 
VariableV2: CPU 
Scatte

Param(parent='Pipeline_ada025e49033', name='stages', doc='a list of pipeline stages')

In [20]:
loadedPipeline\
  .fit(dfTest)\
  .transform(dfTest)\
  .select("ner.result")\
  .show(2, False)

+-------------------------------------------------------------------------------------------------------+
|result                                                                                                 |
+-------------------------------------------------------------------------------------------------------+
|[O, O, O, B-PER, O, O, O, O, B-LOC, O, B-LOC, I-LOC]                                                   |
|[B-PER, I-PER, O, O, O, O, O, O, O, B-LOC, I-LOC, O, B-PER, I-PER, O, O, O, O, O, O, O, O, B-LOC, O, O]|
+-------------------------------------------------------------------------------------------------------+

